# Project Luther: Soccer Players Values

Elvis Davalillo - info@elvisdavalillo.com

Part 1/2 - Scrapping, organization, and cleaning for soccer players data

Scrapped website: www.sofifa.com

Process:

    1.Webscraping basic player information from main subpage 
    2.Webscraping detailed player information from every player subpage
    3.Cleaning the data

In [ ]:
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import random
import urllib.request
import requests
import warnings
import time
import random
from __future__ import print_function, division
from bs4 import BeautifulSoup

In [ ]:
# import requests
requests.__path__

In [ ]:
base_url = 'https://sofifa.com/players?offset='
response = requests.get(base_url)

In [ ]:
response.status_code # status code = 200 => OK

In [ ]:
page = response.text

In [ ]:
soup = BeautifulSoup(page, "html5")

In [ ]:
print(soup.tbody.prettify());

## Webscrape basic player information from main subpages

In [ ]:
# scrape data from thousands of players going through different offsets=subpages 
offset = 0
columns = ['ID', 'Name', 'Age', 'Country','Position','Overall', 'Potential', 'Team'
           , 'Value', 'Salary']
data = DataFrame(columns=columns)

In [ ]:
# start first with the basic player data from the url https://sofifa.com/players?offset=
for offset in range(0,300):
    offset+=1
    url = base_url + str(offset*61)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    table_body = soup.find('tbody')
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        player_id = td[0].find('img').get('id').strip()
        name = td[1].findAll('a')[1].text
        age = td[2].text.strip()
        country = td[1].find('a').get('title')
        position = td[1].findAll('a')[2].text
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].findAll('a')[0].text
        value = td[7].text.strip()
        salary = td[8].text
        player_data = DataFrame([[player_id, name, age, country, position, overall, 
                                  potential, club, value, salary]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
    data.to_csv('baseline_data.csv', encoding='utf-8')
    if (offset % 20 == 0):
        print(offset)

In [69]:
data = pd.read_csv('baseline_data.csv')
player = None
player_url = 'https://sofifa.com/player/'
id_list = data['ID']
count = 0
data.head(5)

,Unnamed: 0,ID,Name,Age,Country,Position,Overall,Potential,Team,Value,Salary
0,0,236007,E. Barco,19,Argentina,LM,74,89,Atlanta United,€12M,€4K
1,1,229582,G. Mancini,22,Italy,CB,74,85,Atalanta,€8.5M,€18K
2,2,232104,D. James,20,Wales,LM,70,82,Swansea City,€3.2M,€12K
3,3,235883,R. Sessegnon,18,England,LM,75,86,Fulham,€10.5M,€23K
4,4,224232,N. Barella,21,Italy,CM,79,89,Cagliari,€20M,€28K


In [70]:
# let's filter the player list to only include players from the largest 3 European Leagues
teams_by_league = pd.read_csv('teams_by_league.csv')

In [71]:
teams_by_league = teams_by_league.iloc[:,:2]

In [72]:
teams_filter = list(teams_by_league.Team) 

In [73]:
small_data = data[data['Team'].isin(teams_filter)].reset_index()
small_data = small_data.drop(['index','Unnamed: 0'],axis=1) 

In [74]:
# Add the league column so we can reference to it later
small_data = pd.merge(small_data,teams_by_league[['Team','League']],on='Team', how='left')

In [77]:
small_data.to_csv('small_player_data.csv', encoding='utf-8')

In [78]:
# Now let's gather the skills statistics for each player

In [79]:
skill_names = []

## Webscrape detailed player information from player subpage

In [ ]:
player_data_url = 'https://sofifa.com/player/'
r = 0
for index, row in small_data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'column col-4'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            #print(a)
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    attr_data = DataFrame(columns=skill_names)
    for key in skill_map.keys():
        attr_data.loc[r,key] = skill_map[key]
    r = r + 1
    master_data = pd.concat([master_data, attr_data])
    if r % 500 == 0:
        print(r)
        time.sleep(2)
        master_data.to_csv('skills_data.csv', encoding='utf-8')

In [80]:
master_data = DataFrame(columns=skill_names)

In [81]:
skills_data = pd.read_csv('skills_data.csv')

## Data Cleaning

In [82]:
skills_data = skills_data.drop(['Unnamed: 0'],axis=1) 

In [83]:
skills_data.shape

(2000, 35)

In [84]:
# let's make ID the index for our DF to be able to merge the master_data set with the small_data set
skills_data = skills_data.set_index('ID')

In [85]:
small_data = small_data.set_index('ID')

In [86]:
# let's combine the basic data and the skills of the players 

In [87]:
full_data = pd.merge(small_data,skills_data, left_index=True, right_index=True)

In [88]:
full_data.describe()

,Age,Overall,Potential
count,2004.000000,2004.000000,2004.000000
mean,24.943613,72.365768,77.750000
std,5.076973,8.054014,5.715494
min,16.000000,47.000000,52.000000
25%,21.000000,66.000000,74.000000
50%,25.000000,74.000000,78.000000
75%,29.000000,78.000000,82.000000
max,43.000000,94.000000,94.000000


In [89]:
full_data.columns

Index(['Name', 'Age', 'Country', 'Position', 'Overall', 'Potential', 'Team',
       'Value', 'Salary', 'League', 'Acceleration', 'Aggression', 'Agility',
       'Balance', 'Ball Control', 'Composure', 'Crossing', 'Curve',
       'Dribbling', 'FK Accuracy', 'Finishing', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Heading Accuracy',
       'Interceptions', 'Jumping', 'Long Passing', 'Long Shots', 'Marking',
       'Penalties', 'Positioning', 'Reactions', 'Short Passing', 'Shot Power',
       'Sliding Tackle', 'Sprint Speed', 'Stamina', 'Standing Tackle',
       'Strength', 'Vision', 'Volleys'],
      dtype='object')

In [90]:
# find column names with nan values 
null_columns = full_data.columns[full_data.isnull().any()]
full_data[null_columns].isnull().sum()

Agility           11
Balance           11
Composure         72
Curve             11
Jumping           11
Sliding Tackle    11
Vision            11
Volleys           11
dtype: int64

In [91]:
# find the rows that had nan values
full_data[full_data.isnull().any(axis=1)].loc[:, ["Agility", "Balance", "Composure", "Curve",
                                                  "Jumping", "Sliding Tackle", "Vision","Volleys"]]

,Agility,Balance,Composure,Curve,Jumping,Sliding Tackle,Vision,Volleys
ID,,,,,,,,
185,45,43,NaN,12,74,12,34,14
241,72,70,NaN,87,59,51,83,75
246,68,82,NaN,74,64,36,91,85
450,75,80,NaN,68,82+9,24,76,73
1041,55,90,NaN,60,49,69,69,60
1109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,53,65,NaN,56,74,19,57,80+2
1419,66,79,NaN,65-2,78,78-3,79-1,62


In [92]:
#remove the rows with a large number of NaN
full_data = full_data.drop([1109,1183,1615,4738,5003,5589,6235,9512,146641]);

In [93]:
#fill the NaN with the feature average
full_data = full_data.fillna(full_data.mean())

In [94]:
#fill remaining NaN with zeros
full_data = full_data.fillna(0)

In [95]:
full_data['Value'],full_data['Salary'] = full_data['Value'].str.replace('€',''),full_data['Salary'].str.replace('€','')

In [96]:
#make the value column an integer number with the same unit across observations
full_data.Value = (full_data.Value.replace(r'[KM]+$', '', regex=True).astype(float) * \
        full_data.Value.str.extract(r'[\d\.]+([KM]+)', expand=False)
        .fillna(1)
        .replace(['K','M'], [10**3, 10**6]).astype(int))

In [97]:
full_data

,Name,Age,Country,Position,Overall,Potential,Team,Value,Salary,League,...,Reactions,Short Passing,Shot Power,Sliding Tackle,Sprint Speed,Stamina,Standing Tackle,Strength,Vision,Volleys
ID,,,,,,,,,,,,,,,,,,,,,
185,D. James,41,England,GK,65,65,Bournemouth,0.0,0,English Premier League,...,64,26,23,12,45,52,11,75,34,14
241,R. Giggs,39,Wales,LM,78,78,Manchester United,0.0,75K,English Premier League,...,76,85,71,51,60,48,43,61,83,75
246,P. Scholes,37,England,CM,80,80,Manchester United,0.0,0,English Premier League,...,82,89,88,36,53,34,49,63,91,85
450,D. Vassell,31,England,ST,69,69,Leicester City,0.0,0,English Premier League,...,67,54,76,24,83,74,26,69+3,76,73
768,Felipe,33,Brazil,CB,75,75,SPAL,2700000.0,13K,Italian Serie A,...,72,62,68,76,41,48,80,67,46,19
1041,J. Zanetti,39,Argentina,RM,75,75,Inter,0.0,0,Italian Serie A,...,80,84,71,69,69,55,75,77,69,60
1178,P. Cannavaro,36,Italy,CB,75,75,Sassuolo,900000.0,30K,Italian Serie A,...,66,62,72,75,31,36,76,75,50,33
1198,F. Inzaghi,37,Italy,ST,73,73,Milan,0.0,0,Italian Serie A,...,88,61,55-13,19,39-2,31,13,30,57,80+2
1238,F. Totti,39,Italy,CF,80,80,Roma,1500000.0,50K,Italian Serie A,...,79,90,88,10,34,54,24,75,91,90


In [98]:
#make the salary column an integer number with the same unit across observations
full_data.Salary = (full_data.Salary.replace(r'[KM]+$', '', regex=True).astype(float) * \
        full_data.Salary.str.extract(r'[\d\.]+([KM]+)', expand=False)
        .fillna(1)
        .replace(['K','M'], [10**3, 10**6]).astype(int))

In [99]:
numerical_columns_1 = ['Acceleration', 'Aggression', 'Agility', 'Balance',
       'Ball Control', 'Composure', 'Crossing', 'Curve', 'Dribbling',
       'FK Accuracy', 'Finishing', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes', 'Heading Accuracy', 'Interceptions',
       'Jumping', 'Long Passing', 'Long Shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short Passing', 'Shot Power',
       'Sliding Tackle', 'Sprint Speed', 'Stamina', 'Standing Tackle',
       'Strength', 'Vision', 'Volleys']

In [100]:
#before converting the features to integers we need to get rid of "-" in the data
for skill in numerical_columns_1:
    full_data[skill] = full_data[skill].map(lambda x: str(x)[:2])

In [101]:
#Let's convert the numerical columns to integers
numerical_columns_2 = ['Age','Overall', 'Potential',
       'Value', 'Salary', 'Acceleration', 'Aggression', 'Agility', 'Balance',
       'Ball Control', 'Composure', 'Crossing', 'Curve', 'Dribbling',
       'FK Accuracy', 'Finishing', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes', 'Heading Accuracy', 'Interceptions',
       'Jumping', 'Long Passing', 'Long Shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short Passing', 'Shot Power',
       'Sliding Tackle', 'Sprint Speed', 'Stamina', 'Standing Tackle',
       'Strength', 'Vision', 'Volleys']

In [102]:
#converting numerical columns to int
for column in numerical_columns_2:
    full_data[column] = full_data[column].map(lambda x: int(x))

In [103]:
full_data.dtypes

Name                object
Age                  int64
Country             object
Position            object
Overall              int64
Potential            int64
Team                object
Value                int64
Salary               int64
League              object
Acceleration         int64
Aggression           int64
Agility              int64
Balance              int64
Ball Control         int64
Composure            int64
Crossing             int64
Curve                int64
Dribbling            int64
FK Accuracy          int64
Finishing            int64
GK Diving            int64
GK Handling          int64
GK Kicking           int64
GK Positioning       int64
GK Reflexes          int64
Heading Accuracy     int64
Interceptions        int64
Jumping              int64
Long Passing         int64
Long Shots           int64
Marking              int64
Penalties            int64
Positioning          int64
Reactions            int64
Short Passing        int64
Shot Power           int64
S

In [104]:
full_data.to_csv('full_data.csv', encoding='utf-8')